<a href="https://colab.research.google.com/github/RuiaRui/project_CSM/blob/main/Topic_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
if 'google.colab' in sys.modules:
    ! pip install emoji --upgrade
    ! pip install pandas-profiling==2.*
    ! pip install plotly==4.*
    ! python -m spacy download en_core_web_lg
    ! pip install pyldavis
    ! pip install gensim
    ! pip install chart_studio
    ! pip install --upgrade autopep8
!pip install pyyaml==5.4.1

In [2]:
# Required Libraries

#Base and Cleaning 
import json
import requests
import pandas as pd
import numpy as np
import regex
import re
import string
from collections import Counter
import csv

#Visualizations
# import plotly.express as px
# import seaborn as sns
# import matplotlib.pyplot as plt 
# import pyLDAvis.gensim_models as gensimvis
# import chart_studio
# import chart_studio.plotly as py 
# import chart_studio.tools as tls

#Natural Language Processing (NLP)
import spacy
import gensim
from spacy.tokenizer import Tokenizer
from gensim.corpora import Dictionary
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
from gensim.parsing.preprocessing import STOPWORDS as SW
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint
# from wordcloud import STOPWORDS
# stopwords = set(STOPWORDS)

In [3]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Load and Preprocess

In [4]:
import pandas as pd
import csv


path = '/content/drive/MyDrive/0401_UkraineCombinedTweetsDeduped.csv'
tweets_df = pd.read_csv(path, index_col=0,encoding='utf-8', quoting=csv.QUOTE_ALL)
tweets_df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,userid,username,acctdesc,location,following,followers,totaltweets,usercreatedts,tweetid,tweetcreatedts,retweetcount,text,hashtags,language,coordinates,favorite_count,extractedts
0,16882774,Yaniela,"Animal lover, supports those who fight injusti...",Hawaii,1158,392,88366,2008-10-21 07:34:04.000000,1509681950042198030,2022-04-01 00:00:00.000000,3412,⚡The Ukrainian Air Force would like to address...,[],en,NaN,0,2022-04-01 00:44:20.097867
1,3205296069,gregffff,NaN,NaN,122,881,99853,2015-04-25 11:24:34.000000,1509681950151348229,2022-04-01 00:00:00.000000,100,Chernihiv oblast. Ukrainians welcome their lib...,"[{'text': 'russianinvasion', 'indices': [77, 9...",en,NaN,0,2022-04-01 00:09:37.148770
2,1235940869812809728,ThanapornThon17,เล่นไวโอลิน\nพูดภาษาจีน,NaN,231,72,5481,2020-03-06 14:52:01.000000,1509681950683926556,2022-04-01 00:00:00.000000,9,America 🇺🇸 is preparing for something worse th...,"[{'text': 'RussianUkrainianWar', 'indices': [7...",en,NaN,0,2022-04-01 00:09:37.125654
3,1347985375566966784,I_Protest_2021,01000001 01101110 01101111 01101110 01111001 0...,International Web Zone,399,377,301,2021-01-09 19:15:44.000000,1509681951116046336,2022-04-01 00:00:00.000000,573,JUST IN: #Anonymous has hacked &amp; released ...,"[{'text': 'Anonymous', 'indices': [25, 35]}]",en,NaN,0,2022-04-01 00:31:39.674382
4,1505394816636846083,Marsh_Win_01,🌿@Pickaw @TWITTERPICKER 🌿Winning isn’t everyth...,Hunter Account,158,25,8982,2022-03-20 04:04:40.000000,1509681951304990720,2022-04-01 00:00:00.000000,190,***PUBLIC MINT NOW LIVE***\n\nFor \n@billionai...,[],en,NaN,0,2022-04-01 00:09:37.101459


In [6]:
tweets_df=tweets_df[tweets_df['text'].str.contains('refugee')]
tweets_en=tweets_df[tweets_df.language=='en']
tweets_en=tweets_en[['tweetid','text','language']]
tweets_en.shape

(2354, 3)

In [7]:
def remove_urls(x):
    cleaned_string = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', str(x), flags=re.MULTILINE)
    return cleaned_string

# leave out emojis
def deEmojify(x):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'', x)

def remove_symbols(x):
    cleaned_string = re.sub(r"[^a-zA-Z0-9?!.,]+", ' ', x)
    return cleaned_string  

tweets_en['text'] = tweets_en['text'].str.lower()
tweets_en['text'] = tweets_en['text'].apply(remove_urls)
tweets_en['text'] = tweets_en['text'].apply(deEmojify)
tweets_en['text'] = tweets_en['text'].apply(remove_symbols)

tweets_en.text.head(10)

328     here s the link to our therapeutic tale for re...
364     russian nobel peace prize winner sells medal t...
482     i see colmogorman hasn t volunteered to look a...
859     a refugee from mariupol tells how the afu azov...
1004    i m on a longer assignment in warsaw poland co...
1835    here is a very emotional thank you video from ...
1856    the human cost of war as conflict escalates in...
2267    the human cost of war as conflict escalates in...
2518     verstyukivan ... great news from the taprooms...
2576     next 9 april the eu will become one with ukra...
Name: text, dtype: object

## LDA

In [8]:
! python -m spacy download en

     |████████████████████████████████| 12.0 MB 5.2 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [9]:
nlp = spacy.load('en_core_web_sm')

### Tokenizing

In [10]:
# Tokenizer
tokenizer = Tokenizer(nlp.vocab)


# Custom stopwords
custom_stopwords = ['hi','\n','\n\n', '&amp;', ' ', '.', '-', 'got', "it's", 'it’s', "i'm", 'i’m', 'im', 'want', 'like', '$', '@']

stopword = stopwords.words('english')
# Customize stop words by adding to the default list
STOP_WORDS = nlp.Defaults.stop_words.union(custom_stopwords)

# ALL_STOP_WORDS = spacy + gensim + wordcloud
ALL_STOP_WORDS = STOP_WORDS.union(SW).union(stopword)


tokens = []

for doc in tokenizer.pipe(tweets_en['text'], batch_size=500):
    doc_tokens = []    
    for token in doc: 
        if token.text.lower() not in STOP_WORDS:
            doc_tokens.append(token.text.lower())   
    tokens.append(doc_tokens)

# Makes tokens column
tweets_en['tokens'] = tokens

In [11]:
tweets_en['tokens_back_to_text'] = [' '.join(map(str, l)) for l in tweets_en['tokens']]


In [36]:
tweets_en.head()
tweets_en=tweets_en.drop_duplicates(subset=['text'])

In [37]:
# Create a id2word dictionary
id2word = Dictionary(tweets_en['tokens'])
print(len(id2word))
# Filtering Extremes
id2word.filter_extremes(no_below=2, no_above=.99)
print(len(id2word))
# Creating a corpus object 
corpus = [id2word.doc2bow(d) for d in tweets_en['tokens']]
# Instantiating a Base LDA model 
base_model = LdaMulticore(corpus=corpus, num_topics=10, id2word=id2word, workers=12, passes=5)
# Filtering for words 
words = [re.findall(r'"([^"]*)"',t[1]) for t in base_model.print_topics()]
# Create Topics
topics = [' '.join(t[0:10]) for t in words]
# Getting the topics
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

6081
2013
------ Topic 0 ------
refugees ukraine ukrainian people s million russia amp ukraine. support

------ Topic 1 ------
refugees ukrainian s ukraine eu refugees. standwithukraine poland refugee people

------ Topic 2 ------
ukraine refugees s refugee russia nato ukrainian uk eu families

------ Topic 3 ------
ukraine refugees ukrainian help war amp s refugee standwithukraine poland

------ Topic 4 ------
ukraine refugees amp poland. people ukrainian standwithukraine support war refugees,

------ Topic 5 ------
ukraine refugees s refugee amp help children world war t

------ Topic 6 ------
ukraine refugees ukrainian binance trc20 wallet tsacpqsmrbenjm2axr6crsm1jfeyxqtwkv donate russia crypto

------ Topic 7 ------
ukraine refugees refugee ukrainian s russia ukraine. help war standwithukraine

------ Topic 8 ------
ukraine refugees ukrainian amp refugee standwithukraine refugees. support help war

------ Topic 9 ------
refugees ukraine amp support humanitarian s help need russia w

In [34]:
# Getting the topics
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
ukraine refugees fleeing country. poland children border war refugee 300,000

------ Topic 1 ------
refugees ukraine uk visas refugees. amp ukrainian standwithukraine 10 office

------ Topic 2 ------
ukraine ukrainian refugees war support refugees. standwithukraine countries donate humanitarian

------ Topic 3 ------
refugees ukraine people support amp refugee ukrainian countries uk help

------ Topic 4 ------
refugees ukraine s russia ukrainian people help amp standwithukraine refugee



## BERTopic

In [ ]:
! pip install bertopic
! pip install pynndescent==0.5

In [14]:
#Base and Cleaning 
import json
import requests
import pandas as pd
import numpy as np
import regex
import re
import string
from collections import Counter
import csv

In [39]:
tweets_en=tweets_en.reset_index()
tweets_en=tweets_en.drop(['index'],axis=1)

In [40]:
tweets_en

,tweetid,text,language,tokens,tokens_back_to_text
0,1509682278514970628,here s the link to our therapeutic tale for re...,en,"[s, link, therapeutic, tale, refugee, children...",s link therapeutic tale refugee children ukrai...
1,1509682320630026241,russian nobel peace prize winner sells medal t...,en,"[russian, nobel, peace, prize, winner, sells, ...",russian nobel peace prize winner sells medal f...
2,1509682449042878464,i see colmogorman hasn t volunteered to look a...,en,"[colmogorman, hasn, t, volunteered, look, afte...","colmogorman hasn t volunteered look after, say..."
3,1509682961377120257,a refugee from mariupol tells how the afu azov...,en,"[refugee, mariupol, tells, afu, azov, fired, c...",refugee mariupol tells afu azov fired car chil...
4,1509683141841096718,i m on a longer assignment in warsaw poland co...,en,"[m, longer, assignment, warsaw, poland, coveri...",m longer assignment warsaw poland covering ref...
...,...,...,...,...,...
873,1510038359049220110,the jewish federations of north america are la...,en,"[jewish, federations, north, america, launchin...",jewish federations north america launching kin...
874,1510038458378694662,thank you fcdogovuk trussliz and the british p...,en,"[thank, fcdogovuk, trussliz, british, people.,...",thank fcdogovuk trussliz british people. new e...
875,1510039501359370240,the russian ongoing aggression against ukraine...,en,"[russian, ongoing, aggression, ukraine, city, ...",russian ongoing aggression ukraine city malaya...
876,1510041095811612690,president andrzejduda amp first lady of poland...,en,"[president, andrzejduda, amp, lady, poland, ag...",president andrzejduda amp lady poland agata ko...


In [17]:
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

In [43]:
from bertopic import BERTopic

model = BERTopic(verbose=True,
                 #embedding_model='paraphrase-MiniLM-L3-v2', 
                 min_topic_size=5)

In [44]:
headline_topics, probs = model.fit_transform(tweets_en.tokens_back_to_text)

Batches:   0%|          | 0/28 [00:00<?, ?it/s]

2022-04-29 10:25:26,042 - BERTopic - Transformed documents to Embeddings
2022-04-29 10:25:32,460 - BERTopic - Reduced dimensionality with UMAP
2022-04-29 10:25:32,508 - BERTopic - Clustered UMAP embeddings with HDBSCAN


In [45]:
model.get_topic_info()

,Topic,Count,Name
0,-1,292,-1_refugees_ukraine_refugee_russia
1,0,115,0_help_ukrainian_children_war
2,1,76,1_poland_warsaw_refugees_ukraine
3,2,69,2_moldova_children_war_ukraine
4,3,63,3_crypto_usdt_network_binance
5,4,17,4_episode_listen_discuss_gamaliy
6,5,16,5_visa_uk_britain_visas
7,6,16,6_support_eu_easier_quicker
8,7,14,7_students_leadership_study_find
9,8,13,8_racism_african_racial_bias


In [46]:
model.visualize_topics()

In [47]:
model.visualize_barchart()